In [ ]:
!pip install tweet-preprocessor

In [ ]:
!pip install nltk

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install tweet-preprocessor

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install nltk

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import preprocessor as p
import re
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
import string
import html
import requests
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
#import xgboost as xgb
#from google.colab import files

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Liyuannnnn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Liyuannnnn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
uploaded = files.upload()

In [2]:
df = pd.read_csv('sentiment_analysis.csv')
df = df.drop(['ID'], axis=1)
df.head()

,text,label
0,Josh Jenkins is looking forward to TAB Breeder...,1
1,RT @MianUsmanJaved: Congratulations Pakistan o...,1
2,"RT @PEPalerts: This September, @YESmag is taki...",1
3,"RT @david_gaibis: Newly painted walls, thanks ...",1
4,RT @CedricFeschotte: Excited to announce: as o...,1


In [3]:
def clean_text(_sentence):

    #removes emojis, pictographs, map symbols and flags
    emoji_pattern = re.compile(pattern = "["
          u"\U0001F600-\U0001F64F" 
          u"\U0001F300-\U0001F5FF" 
          u"\U0001F680-\U0001F6FF" 
          u"\U0001F1E0-\U0001F1FF" 
                            "]+", flags = re.UNICODE)
    _sentence = emoji_pattern.sub(r'',_sentence)

    #emoves tags and attributes
    _sentence = re.sub('(<[^>]*>)','',_sentence)

    #converts character code to string
    _sentence = html.unescape(_sentence)

    #removes urls
    _sentence = re.sub("(\w+:\/\/\S+)|(http[^\s]+)|(www.[^\s]+)",'',_sentence, flags=re.MULTILINE)
    
    #removes RT
    _sentence = re.sub("RT",'',_sentence)
    
    #converts the sentence to a lower case
    _sentence = _sentence.lower()

    #removes punctuation
    _sentence = re.sub(r'\@\w+|\#\w+','', _sentence)
    _sentence = _sentence.translate(str.maketrans('', '', (string.punctuation)))
    
    #checks for stop words
    stop_words = set(stopwords.words('english'))
    _sentence = [word for word in str(_sentence).split() if word not in stop_words]

    #joins the word and returns a sentence
    return " ".join(_sentence)

#function to preprocess tweets
def preprocess_tweets(tweets_df):

    #apply clean text in a vectorized format
    vfunc_clean_text = np.vectorize(clean_text)
    tweets_df['text'] = vfunc_clean_text(tweets_df['text']) #create a new column that consists of preprocessed tweets
    
    return tweets_df

In [4]:
df = preprocess_tweets(df)

In [5]:
df

,text,label
0,josh jenkins looking forward tab breeders crow...,1
1,congratulations pakistan becoming world odds,1
2,september taking maine mendoza’s surprise than...,1
3,newly painted walls thanks million custodial p...,1
4,excited announce july 2017 feschotte lab reloc...,1
...,...,...
550386,cant stop watching hermm,0
550387,poor old tom odell doesnt look like would know...,0
550388,smashed 7 ants awesome gamehjfjfi,1
550389,morning girls wonderful,1


In [6]:
X = df.loc[:, ['text']]
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

In [ ]:
vectorizer = CountVectorizer(binary=False,ngram_range=(1,1))
X_train_ = vectorizer.fit_transform(X_train['text'])
X_test_ = vectorizer.transform(X_test['text'])
print (X_train.shape, X_test.shape) 

TV = TfidfVectorizer(max_features = 1000)   
X_train = TV.fit_transform(X_train['text'])
X_test = TV.fit_transform(X_test['text'])
print(np.shape(X_train))
print(np.shape(X_test)) 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
def model_training(model, X_train, y_train, X_test, y_test):
    model.fit(X_train,y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)      
  
    acc_train = metrics.accuracy_score(y_train, y_pred_train)
    acc_test = metrics.accuracy_score(y_test, y_pred_test)

    #Print Accuracies for comparison
    print("The accuracy score for the Train set is: {}".format(acc_train))
    print("The accuracy score for the Test set is: {}".format(acc_test))
    print(classification_report(y_test, y_pred_test))
    print(confusion_matrix(y_test, y_pred_test, labels=[0, 1]))
  
  

In [ ]:
KNN_model = KNeighborsClassifier()
print("Results when KNN model is used:\n")
model_training(KNN_model, X_train_, y_train,  X_test_, y_test)

In [ ]:
LR_model = LogisticRegression()
print("Results when Logistic Regression is used:\n")
model_training(LR_model, X_train_, y_train,  X_test_, y_test)

In [ ]:
SVC_model = LinearSVC()
print("Results when SVC model is used:\n")
model_training(SVC_model, X_train_, y_train,  X_test_, y_test)

In [ ]:
MultinomialNB_model = MultinomialNB()
print("Results when Naive Bayes model is used:\n")
model_training(MultinomialNB_model, X_train_, y_train,  X_test_, y_test)

In [ ]:
randomforest_model = RandomForestClassifier()
print("Results when Random Forest model is used:\n")
model_training(randomforest_model, X_train_, y_train,  X_test_, y_test)

In [ ]:
decisiontree_model = DecisionTreeClassifier()
print("Results when Decision Tree model is used:\n")
model_training(decisiontree_model, X_train_, y_train,  X_test_, y_test)

Logistic

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
bag = count.fit_transform(X_train['text'].tolist())

count.vocabulary_

{'try': 127454,
 'new': 87426,
 'chicken': 28054,
 'nugget': 89439,
 'dipped': 38163,
 'favorite': 46744,
 'dipping': 38168,
 'sauce': 108708,
 'taste': 121769,
 'great': 54838,
 'enjoy': 43153,
 'latest': 72774,
 'album': 10490,
 'getting': 52516,
 'books': 21782,
 'literacy': 74932,
 'lol': 75520,
 'things': 123774,
 'get': 52483,
 'hilarious': 59485,
 'involved': 64821,
 'ignore': 62550,
 'wait': 132892,
 'faux': 46712,
 'outrage': 92516,
 'hypocrisy': 61928,
 'bite': 20398,
 'arse': 13947,
 'brazilian': 22707,
 'soccer': 115227,
 'team': 122123,
 'plane': 96954,
 'crashes': 32779,
 'colombia': 30313,
 '76': 6725,
 'dead': 35754,
 'noah': 88415,
 'exceeded': 44803,
 'expectations': 45152,
 'please': 97209,
 'donate': 39346,
 'ly': 77034,
 'wish': 135673,
 'video': 131781,
 '11yr': 1160,
 'old': 90778,
 'us': 130373,
 'cracking': 32688,
 'clarinet': 29141,
 'cases': 26270,
 'making': 77983,
 'fart': 46528,
 'noises': 88526,
 'ugh': 128750,
 'make': 77922,
 'guide': 55615,
 'stupid': 

In [8]:
import numpy as np

from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',
                         smooth_idf=True)

np.set_printoptions(precision=2)

# Feed the tf-idf transformer with our previously created Bag of Words
bag_trans = tfidf.fit_transform(bag).toarray()

In [9]:
from collections import Counter

vocab = Counter()
for twit in X_train.text:
    for word in twit.split(' '):
        vocab[word] += 1

vocab.most_common(20)

[('love', 42385),
 ('happy', 40825),
 ('great', 23262),
 ('good', 21919),
 ('im', 20543),
 ('best', 20086),
 ('amazing', 19574),
 ('day', 18533),
 ('birthday', 17562),
 ('one', 13723),
 ('thank', 13584),
 ('found', 13340),
 ('dont', 12588),
 ('like', 12383),
 ('beautiful', 12258),
 ('transponder', 11427),
 ('much', 11133),
 ('get', 10924),
 ('today', 10857),
 ('cant', 10325)]

In [10]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [11]:
import math

def plot_distribution(vocabulary):

    hist, edges = np.histogram(list(map(lambda x:math.log(x[1]),vocabulary.most_common())), density=True, bins=500)

    p = figure(tools="pan,wheel_zoom,reset,save",
               toolbar_location="above",
               title="Word distribution accross all twits")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555", )
    show(p)

plot_distribution(vocab)

In [12]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Liyuannnnn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

vocab_reduced = Counter()
for w, c in vocab.items():
    if not w in stop:
        vocab_reduced[w]=c

vocab_reduced.most_common(20)

plot_distribution(vocab_reduced)

In [14]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)


param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l2', 'none'],
               'clf__C': [1.0, 10.0, 100.0],
              'clf__solver': ['lbfgs'],
              'clf__max_iter': [1000]},
              
              {'vect__ngram_range': [(1, 1)],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1','l2'],
               'clf__C': [1.0, 10.0, 100.0],
              'clf__solver': ['liblinear'],
              'clf__max_iter': [1000]},
              
              {'vect__ngram_range': [(1, 1)],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l2', 'none'],
               'clf__C': [1.0, 10.0, 100.0],
              'clf__solver': ['newton-cg'],
              'clf__max_iter': [1000]},
              
              {'vect__ngram_range': [(1, 1)],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l2', 'none'],
               'clf__C': [1.0, 10.0, 100.0],
              'clf__solver': ['sag'],
              'clf__max_iter': [1000]}
             ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1)

In [18]:
# Note: This process took 25 hours
gs_lr_tfidf.fit(X_train.values.ravel(), y_train.values.ravel())

Fitting 5 folds for each of 48 candidates, totalling 240 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0))]),
             param_grid=[{'clf__C': [1.0, 10.0, 100.0], 'clf__max_iter': [1000],
                          'clf__penalty': ['l2', 'none'],
                          'clf__solver': ['lbfgs'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__tokenizer': [<function tokenizer at 0x7f8f9967bc10>,
                                              <function tokenizer_po...
                          'vect__tokenizer': [<function tokenizer at 0x7f8f9967bc10>,
                                              <function tokenizer_porter at 0x7f8f9967b940>]},
                         {'clf__C': [1.0, 10.0, 100.0], 'clf__max_iter': [1000],
                          'clf__penalty': ['l2', 'none'],
      

In [19]:
print('Best parameter set: ' + str(gs_lr_tfidf.best_params_))
print('Best accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'clf__C': 10.0, 'clf__max_iter': 1000, 'clf__penalty': 'l2', 'clf__solver': 'liblinear', 'vect__ngram_range': (1, 1), 'vect__tokenizer': <function tokenizer at 0x7f8f9967bc10>}
Best accuracy: 0.974


In [28]:
clf = gs_lr_tfidf.best_estimator_
#clf.fit(X_train.values, y_train)
clf.fit(X_test['text'].tolist(), y_test)
print('Accuracy in test: %.3f' % clf.score(X_test['text'].tolist(), y_test))

Accuracy in test: 0.992
